# NEXAFS Spectral Analysis

This notebook demonstrates how to use the spectral analysis utilities to:
1. Load calculated DFT data from StoBe
2. Apply energy correction using the IGOR method
3. Generate energy-dependent broadened spectra
4. Visualize and analyze the results

The code is based on the `spectral_analysis.py` script, converted to an interactive notebook format.

## 1. Import Required Libraries

First, let's import all the necessary libraries and configure paths.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import xarray as xr
from pathlib import Path
from typing import Dict, Tuple, List, Any, Optional

# Set the project root correctly to the current directory
project_root = Path().resolve()  # This will be /home/hduva/projects/dft-learn
print(f"Project root: {project_root}")
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import utilities for spectral analysis
from src.dftlearn.io.stobe import read_calculations
from src.dftlearn.spectral import (
    calculate_all_spectra,
    plot_combined_spectrum,
    plot_site_comparison,
    plot_site_components,
    plot_width_function,
    compare_broadening_methods,
    save_processed_spectra,
)

# Configure matplotlib for better visualization
%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

Project root: /home/hduva/projects/dft-learn


## 2. Configuration Parameters

Instead of using command-line arguments, we'll define parameters directly in the notebook. You can adjust these parameters as needed.

In [2]:
# Input and output directories
input_dir = Path(project_root) / "output"  # Directory containing calculation outputs
output_dir = Path(project_root) / "results"  # Directory to save results
output_dir.mkdir(
    exist_ok=True, parents=True
)  # Create output directory if it doesn't exist

# Spectral parameters
experimental_edge = 284.5  # Experimental edge energy in eV
emin = 280.0  # Minimum energy for spectrum in eV
emax = 320.0  # Maximum energy for spectrum in eV
npoints = 2000  # Number of points in energy grid

# Energy-dependent broadening parameters
width1 = 0.5  # FWHM at lower energy threshold in eV
width2 = 12.0  # FWHM at upper energy threshold in eV
ewid1 = 288.0  # Lower energy threshold for FWHM transition in eV
ewid2 = 320.0  # Upper energy threshold for FWHM transition in eV

# Processing options
use_variable_width = True  # Use variable width broadening instead of fixed width
use_igor_correction = True  # Use IGOR-style energy correction
save_data = False  # Save processed spectral data to NPZ file

# Derived parameters
energy_range = (emin, emax, npoints)
fwhm_params = {"ewid1": ewid1, "ewid2": ewid2, "width1": width1, "width2": width2}

# Display configuration
print(f"Input directory: {input_dir}")
print(f"Output directory: {output_dir}")
print(f"Experimental edge: {experimental_edge} eV")
print(f"Energy range: {emin} to {emax} eV ({npoints} points)")
print(f"Variable width broadening: {use_variable_width}")
print(f"IGOR-style energy correction: {use_igor_correction}")

Input directory: /home/hduva/projects/dft-learn/output
Output directory: /home/hduva/projects/dft-learn/results
Experimental edge: 284.5 eV
Energy range: 280.0 to 320.0 eV (2000 points)
Variable width broadening: True
IGOR-style energy correction: True


## 3. Load Calculation Data

Now we'll load the calculation data from the StoBe output files.

In [3]:
# Load calculation data
print(f"Loading calculation data from {input_dir}...")
try:
    data_tree = read_calculations(
        input_dir,
        comment="Loaded for spectral analysis",
        save_checkpoint=False,  # No need to save a checkpoint
    )
    print("Data loaded successfully!")
except Exception as e:
    print(f"Error loading data: {str(e)}")
    raise

# Examine the data structure
print("\nData structure:")
data_tree

Loading calculation data from /home/hduva/projects/dft-learn/output...
Data loaded successfully!

Data structure:
Data loaded successfully!

Data structure:


<xarray.DataTree>
Group: /
│   Attributes:
│       date:              2025-04-15
│       description:       Combined data from several calculations extracted data
│       source_directory:  /home/hduva/projects/dft-learn/output
│       version:           1.1
│       comment:           Loaded for spectral analysis
├── Group: /GEOMETRY
│       Dimensions:          (excitation_atom: 4, atom: 57)
│       Coordinates:
│         * atom             (atom) object 456B 'C01' 'C02' 'C03' ... 'N07' 'N08' 'Zn01'
│         * excitation_atom  (excitation_atom) object 32B 'C1' 'C2' 'C3' 'C4'
│       Data variables:
│           x                (excitation_atom, atom) float64 2kB 1.097 0.6501 ... 0.0
│           y                (excitation_atom, atom) float64 2kB -2.796 -4.188 ... 0.0
│           z                (excitation_atom, atom) float64 2kB -0.1269 -0.1554 ... 0.0
│           q                (excitation_atom, atom) float64 2kB 6.0 4.0 ... 7.0 30.0
│           nuc              (excitation_atom, atom) int64 2kB 6 6 6 6 6 ... 7 7 7 7 30
│           mass             (excitation_atom, atom) float64 2kB 12.01 12.01 ... 65.37
│           neq              (excitation_atom, atom) object 2kB '1/' '2/' ... '57/'
│           grid             (excitation_atom, atom) int64 2kB 1 1 1 1 1 1 ... 1 1 1 1 1
│           grp              (excitation_atom, atom) int64 2kB 32 32 32 32 ... 32 32 32
├── Group: /BASIS
│       Dimensions:           (excitation_atom: 4, atom: 48)
│       Coordinates:
│         * atom              (atom) object 384B 'C01' 'C02' 'C03' ... 'H14' 'H15' 'H16'
│         * excitation_atom   (excitation_atom) object 32B 'C1' 'C2' 'C3' 'C4'
│       Data variables:
│           orbital_basis     (excitation_atom, atom) object 2kB 'O-CARBON iii_iglo' ...
│           auxiliary_basis   (excitation_atom, atom) object 2kB 'A-CARBON (5,2;5,2)'...
│           model_core_basis  (excitation_atom, atom) object 2kB None ... None
├── Group: /GND
│   ├── Group: /GND/energy
│   │       Dimensions:                   (excitation_atom: 4)
│   │       Coordinates:
│   │         * excitation_atom           (excitation_atom) object 32B 'C1' 'C2' 'C3' 'C4'
│   │       Data variables:
│   │           total_energy              (excitation_atom) float64 32B -6.636e+04 ... -6...
│   │           nuc-nuc_energy            (excitation_atom) float64 32B 8.772e+04 ... 8.7...
│   │           el-nuc_energy             (excitation_atom) float64 32B -3.334e+05 ... -3...
│   │           kinetic_energy            (excitation_atom) float64 32B 6.595e+04 ... 6.5...
│   │           coulomb_energy            (excitation_atom) float64 32B 1.186e+05 ... 1.1...
│   │           ex-cor_energy             (excitation_atom) float64 32B -5.242e+03 ... -5...
│   │           total_exchange_energy     (excitation_atom) float64 32B -4.996e+03 ... -4...
│   │           total_correlation_energy  (excitation_atom) float64 32B -245.9 ... -245.9
│   │           lumo-energy               (excitation_atom) float64 32B -3.365 ... -3.36
│   │           homo-energy               (excitation_atom) float64 32B -3.399 ... -3.393
│   └── Group: /GND/spin
│           Dimensions:           (excitation_atom: 4, orbital: 727)
│           Coordinates:
│             * orbital           (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727
│             * excitation_atom   (excitation_atom) object 32B 'C1' 'C2' 'C3' 'C4'
│           Data variables:
│               homo              (excitation_atom) int64 32B 116 116 116 116
│               lumo              (excitation_atom) int64 32B 117 117 117 117
│               alpha_occupation  (excitation_atom, orbital) float64 23kB 1.0 1.0 ... 0.0
│               alpha_energy      (excitation_atom, orbital) float64 23kB -9.408e+03 ... ...
│               alpha_symmetry    (excitation_atom, orbital) object 23kB '1A' ... '727A'
│               alpha_position    (excitation_atom, orbital) int64 23kB 1 2 3 ... 726 727
│               beta_occupation   (excitation_atom,

## 4. Process Spectra

Now we'll process the spectra using the loaded data and our configuration parameters.

In [4]:
# Check if transitions data exists and seems valid before processing
print("Checking for transitions data in data_tree...")
transitions_valid = False
required_vars = [
    "energy",
    "oscillator_strength",
    "oslx",
    "osly",
    "oslz",
]  # Variables expected by the function

if "TP" in data_tree and "transitions" in data_tree["TP"]:
    transitions_node = data_tree["TP"]["transitions"]
    print("Found /TP/transitions group.")

    # Check for required variables
    missing_vars = [
        var for var in required_vars if var not in transitions_node.data_vars
    ]
    if not missing_vars:
        print(f"Found required variables: {required_vars}")
        # Optional: Check if oscillator_strength contains only NaNs
        if "oscillator_strength" in transitions_node and np.all(
            np.isnan(transitions_node["oscillator_strength"].values)
        ):
            print(
                "Warning: 'oscillator_strength' contains only NaN values. Processing might fail or produce empty spectra."
            )
            # Decide if processing should continue despite all NaNs
            transitions_valid = (
                True  # Allow processing for now, function might handle it
            )
        else:
            transitions_valid = True
    else:
        print(
            f"Error: Missing required transition variables in /TP/transitions: {missing_vars}"
        )
else:
    print("Error: /TP/transitions group not found in data_tree.")

# Process spectra only if transitions data seems valid
if transitions_valid:
    print("\nProcessing spectra...")
    try:
        spectra = calculate_all_spectra(
            data_tree=data_tree,
            experimental_edge=experimental_edge,
            energy_range=energy_range,
            variable_width=use_variable_width,
            fwhm_params=fwhm_params,
            fixed_fwhm=width1,  # Use width1 as the fixed width value
            use_igor_correction=use_igor_correction,
        )

        # Get site information
        num_sites = len(spectra.get("sites", {}))  # Use .get for safety
        site_keys = list(spectra.get("sites", {}).keys())

        # Print information about the energy corrections
        print(f"Number of sites processed: {num_sites}")
        if num_sites > 0:
            print("Energy corrections:")
            for site, site_data in spectra["sites"].items():
                correction = site_data.get("correction", "N/A")
                print(f"  Site {site}: {correction} eV")
        else:
            print("No site data found in the processed spectra.")

    except ValueError as e:
        print(f"\nError during spectral processing: {e}")
        print(
            "This might indicate an issue within calculate_all_spectra or incompatible data structure/values."
        )
        # Assign default values to allow subsequent cells to run without error
        spectra = {"sites": {}}
        num_sites = 0
        site_keys = []
    except Exception as e:
        print(f"\nAn unexpected error occurred during spectral processing: {e}")
        spectra = {"sites": {}}
        num_sites = 0
        site_keys = []

else:
    print("\nSkipping spectral processing due to missing or invalid transitions data.")
    # Assign default values to allow subsequent cells to run without error
    spectra = {"sites": {}}  # Create a minimal structure
    num_sites = 0
    site_keys = []

Checking for transitions data in data_tree...
Found /TP/transitions group.
Error: Missing required transition variables in /TP/transitions: ['energy']

Skipping spectral processing due to missing or invalid transitions data.


## 5. Visualize Combined Spectrum

First, let's plot the combined spectrum from all sites.

In [ ]:
# Plot combined spectrum
fig1, ax1 = plot_combined_spectrum(
    spectra,
    show_components=True,
    show_edge=True,
    fig_size=(12, 8),
    title="Combined NEXAFS Spectrum with IGOR Energy Correction",
)

# Save the figure if needed
fig1.savefig(output_dir / "combined_spectrum.png", dpi=300)
plt.show()

## 6. Site Comparison (if multiple sites)

If we have multiple excitation sites, let's compare the spectra from each site.

In [ ]:
# Plot site comparison if multiple sites
if num_sites > 1:
    fig2, ax2 = plot_site_comparison(
        spectra,
        fig_size=(12, 8),
        show_edge=True,
        title="NEXAFS Spectra by Excitation Site",
    )

    # Save the figure if needed
    fig2.savefig(output_dir / "site_comparison.png", dpi=300)
    plt.show()
else:
    print("Only one site detected. Skipping site comparison.")

## 7. Components for Each Site

Now let's examine the in-plane and out-of-plane components for each site individually.

In [ ]:
# Plot components for each site
for site in site_keys:
    fig3, ax3 = plot_site_components(
        spectra, site, show_sticks=True, show_edge=True, fig_size=(12, 8)
    )

    # Save the figure if needed
    fig3.savefig(output_dir / f"site_{site}_components.png", dpi=300)
    plt.show()

## 8. Width Function (for Variable Width Broadening)

If using variable width broadening, let's visualize the width function.

In [ ]:
# Plot width function if variable width is used
if use_variable_width:
    try:
        fig4, ax4 = plot_width_function(
            spectra, fig_size=(12, 6), title="Energy-Dependent Broadening Function"
        )

        # Save the figure if needed
        fig4.savefig(output_dir / "width_function.png", dpi=300)
        plt.show()
    except ValueError as e:
        print(f"Warning: Could not plot width function: {str(e)}")
else:
    print("Variable width broadening not used. Skipping width function plot.")

## 9. Compare Broadening Methods

Let's compare the fixed-width and variable-width broadening methods for the first site.

In [ ]:
# Compare broadening methods for the first site
site_key = site_keys[0]
site_data = spectra["sites"][site_key]
if "transitions_data" in site_data:
    fig5, axes5 = compare_broadening_methods(
        site_data["transitions_data"],
        energy_range=energy_range,
        fixed_fwhm=width1,
        variable_fwhm_params=fwhm_params,
        fig_size=(12, 10),
    )

    # Save the figure if needed
    fig5.savefig(output_dir / f"broadening_comparison_{site_key}.png", dpi=300)
    plt.show()
else:
    print("No transitions data available for broadening comparison.")

## 10. Save Processed Data

Optionally, save the processed spectral data to a file for later use.

In [ ]:
# Save the processed data if requested
if save_data:
    output_file = output_dir / "processed_spectra.npz"
    save_processed_spectra(spectra, output_file)
    print(f"Saved processed spectral data to {output_file}")
else:
    print("Skipping data saving. Set save_data=True if you want to save the data.")

## 11. Custom Analysis

You can add additional cells below for custom analysis of the spectra. Some ideas:

- Compare the IGOR-style energy correction with the rigid shift method
- Combine spectra with experimental data
- Perform peak fitting
- Analyze polarization dependence

In [ ]:
# Example: Extract and analyze a specific site's data
if site_keys:
    site_key = site_keys[0]
    site_data = spectra["sites"][site_key]

    # Print details about the site
    print(f"Details for site {site_key}:")
    print(f"  Energy correction: {site_data.get('correction', 'N/A')} eV")
    print(f"  Experimental edge: {site_data.get('experimental_edge', 'N/A')} eV")
    print(f"  Variable width used: {site_data.get('variable_width', 'N/A')}")

    # Access the raw transition data if needed
    if "transitions_data" in site_data:
        transitions = site_data["transitions_data"]
        print("\nTransition data summary:")
        print(transitions)

        # Plot the strongest transitions
        if "stick_data" in site_data:
            energies = site_data["stick_data"]["energies"]
            strengths = site_data["stick_data"]["strengths"]

            # Find the strongest transitions
            mask = ~np.isnan(strengths)
            sorted_idx = np.argsort(strengths[mask])[::-1]  # Sort descending
            top_n = 5  # Show top 5 transitions

            if len(sorted_idx) >= top_n:
                print(f"\nTop {top_n} strongest transitions:")
                for i in range(top_n):
                    idx = sorted_idx[i]
                    print(
                        f"  {i + 1}. Energy: {energies[mask][idx]:.4f} eV, Strength: {strengths[mask][idx]:.6f}"
                    )

## Summary

This notebook has demonstrated how to:
1. Load StoBe calculation data
2. Process the data with IGOR-style energy correction
3. Apply energy-dependent broadening
4. Visualize the resulting NEXAFS spectra

The key features implemented in the spectral analysis include:

- Proper energy correction using `Ecorr = (EXC - GND) - LUMO`
- Variable width broadening with linear interpolation between energy thresholds
- In-plane and out-of-plane component analysis
- Comparison of different broadening methods

You can further customize this notebook to suit your specific analysis needs.